# Week 11: Marketing Performance Analysis & Optimization
# Wednesday Python - Matplotlib Business Storytelling
# Part 2: Advanced Marketing Visualization and Business Storytelling

## Business Context
**Scenario**: You're preparing a presentation for the CMO of Jumia Nigeria. You need to create compelling visualizations that tell a clear story about marketing performance and provide actionable recommendations.

**Focus**: Moving beyond basic charts to create impactful business narratives with data visualizations.

## 1. Advanced Color Psychology and Branding

Learn to use colors strategically to convey meaning and align with Nigerian business aesthetics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Rectangle
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load our marketing data
df = pd.read_csv('datasets/marketing_campaign_data.csv')
df['first_contact_date'] = pd.to_datetime(df['first_contact_date'])
df['won_date'] = pd.to_datetime(df['won_date'])
df['contact_month'] = df['first_contact_date'].dt.to_period('M')

# Define Nigerian business-appropriate color palettes
nigerian_colors = {
    'primary': '#00875A',      # Nigerian Green
    'secondary': '#FFFFFF',    # White
    'accent': '#003E7E',       # Deep Blue
    'success': '#28A745',      # Success Green
    'warning': '#FFC107',      # Warning Yellow
    'danger': '#DC3545',       # Danger Red
    'info': '#17A2B8',         # Info Blue
    'gradient': ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']
}

# Marketing channel specific colors
channel_colors = {
    'social': '#FF6B6B',       # Social - Red (energy, excitement)
    'paid_search': '#4ECDC4',  # Paid Search - Teal (trust, growth)
    'organic_search': '#45B7D1', # Organic - Blue (natural, calm)
    'email': '#96CEB4',        # Email - Green (direct, growth)
    'referral': '#FECA57'      # Referral - Yellow (warm, recommended)
}

# Set matplotlib defaults for business presentations
plt.rcParams['figure.figsize'] = (14, 9)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

print("Color palette loaded successfully!")
print("\nNigerian Business Color Palette:")
for color_name, color_value in nigerian_colors.items():
    if color_name != 'gradient':
        print(f"  {color_name}: {color_value}")

### 1.1 Strategic Color Application - Marketing Channel Performance

In [ ]:
# Calculate comprehensive channel performance metrics
channel_performance = df.groupby('origin').agg({
    'mql_id': 'count',
    'won_date': lambda x: x.notna().sum(),
    'declared_monthly_revenue': 'sum'
}).rename(columns={
    'mql_id': 'total_mqls',
    'won_date': 'converted_deals'
})

channel_performance['conversion_rate'] = (channel_performance['converted_deals'] / 
                                          channel_performance['total_mqls'] * 100)
channel_performance['avg_revenue_per_mql'] = (channel_performance['declared_monthly_revenue'] / 
                                               channel_performance['total_mqls'])

# Create sophisticated multi-metric visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Marketing Channel Performance Analysis - Nigerian E-commerce', 
             fontsize=20, fontweight='bold', y=0.98)

# 1. MQL Volume with strategic colors
channels = channel_performance.index
mql_colors = [channel_colors.get(ch, nigerian_colors['info']) for ch in channels]

bars1 = ax1.bar(channels, channel_performance['total_mqls'], 
                color=mql_colors, alpha=0.8, edgecolor='black', linewidth=1)
ax1.set_title('MQL Volume by Channel', fontsize=14, fontweight='bold', pad=15)
ax1.set_ylabel('Number of MQLs', fontsize=12, fontweight='bold')
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels with thousands separator
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
            f'{int(height):,}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 2. Conversion Rate with color-coded performance
conv_colors = [nigerian_colors['success'] if rate > 10 else 
                nigerian_colors['warning'] if rate > 5 else 
                nigerian_colors['danger'] for rate in channel_performance['conversion_rate']]

bars2 = ax2.bar(channels, channel_performance['conversion_rate'], 
                color=conv_colors, alpha=0.8, edgecolor='black', linewidth=1)
ax2.set_title('Conversion Rate by Channel', fontsize=14, fontweight='bold', pad=15)
ax2.set_ylabel('Conversion Rate (%)', fontsize=12, fontweight='bold')
ax2.axhline(y=5, color='red', linestyle='--', alpha=0.5, label='Minimum Target')
ax2.axhline(y=10, color='green', linestyle='--', alpha=0.5, label='Good Performance')
ax2.legend()
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# Add percentage labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.2,
            f'{height:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 3. Revenue Generation with gradient effect
revenue_data = channel_performance['declared_monthly_revenue'].sort_values(ascending=True)
colors_gradient = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(revenue_data)))

bars3 = ax3.barh(revenue_data.index, revenue_data.values, 
                 color=colors_gradient, alpha=0.8, edgecolor='black', linewidth=1)
ax3.set_title('Revenue Generation by Channel', fontsize=14, fontweight='bold', pad=15)
ax3.set_xlabel('Total Revenue (₦)', fontsize=12, fontweight='bold')
ax3.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'₦{int(x/1000)}K'))
ax3.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels
for bar in bars3:
    width = bar.get_width()
    ax3.text(width + width*0.01, bar.get_y() + bar.get_height()/2,
            f'₦{width/1000:.0f}K', ha='left', va='center', fontweight='bold', fontsize=11)

# 4. Overall Performance Score (Composite)
performance_score = pd.DataFrame({
    'volume_score': channel_performance['total_mqls'] / channel_performance['total_mqls'].max() * 40,
    'conversion_score': channel_performance['conversion_rate'] / channel_performance['conversion_rate'].max() * 30,
    'revenue_score': channel_performance['declared_monthly_revenue'] / channel_performance['declared_monthly_revenue'].max() * 30
})
performance_score['total_score'] = performance_score.sum(axis=1)

score_colors = [nigerian_colors['primary'] if score >= 70 else 
               nigerian_colors['warning'] if score >= 50 else 
               nigerian_colors['danger'] for score in performance_score['total_score']]

bars4 = ax4.bar(channels, performance_score['total_score'], 
                color=score_colors, alpha=0.8, edgecolor='black', linewidth=1)
ax4.set_title('Overall Performance Score', fontsize=14, fontweight='bold', pad=15)
ax4.set_ylabel('Performance Score (0-100)', fontsize=12, fontweight='bold')
ax4.axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='Average')
ax4.axhline(y=70, color='green', linestyle='--', alpha=0.5, label='Excellent')
ax4.legend()
ax4.grid(axis='y', alpha=0.3, linestyle='--')

# Add score labels
for i, (bar, score) in enumerate(zip(bars4, performance_score['total_score'])):
    ax4.text(bar.get_x() + bar.get_width()/2., score + 1,
            f'{score:.1f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

# Print strategic insights
print("\n" + "="*80)
print("STRATEGIC MARKETING CHANNEL ANALYSIS")
print("="*80)
print(f"\n🏆 BEST OVERALL PERFORMER: {performance_score['total_score'].idxmax()}")
print(f"   Score: {performance_score['total_score'].max():.1f}/100")

best_conversion = channel_performance['conversion_rate'].idxmax()
print(f"\n🎯 HIGHEST CONVERSION RATE: {best_conversion}")
print(f"   Rate: {channel_performance.loc[best_conversion, 'conversion_rate']:.1f}%")

best_revenue = channel_performance['declared_monthly_revenue'].idxmax()
print(f"\n💰 HIGHEST REVENUE GENERATOR: {best_revenue}")
print(f"   Revenue: ₦{channel_performance.loc[best_revenue, 'declared_monthly_revenue']:,.0f}")

print(f"\n📈 RECOMMENDATIONS:")
low_performers = performance_score[performance_score['total_score'] < 50].index.tolist()
if low_performers:
    print(f"   • Optimize underperforming channels: {', '.join(low_performers)}")
print(f"   • Double down on high-performers: {', '.join(performance_score.nlargest(2, 'total_score').index.tolist())}")
print("="*80)

## 2. Advanced Storytelling with Annotations and Callouts

Master the art of guiding your audience through the data story with strategic annotations.

In [ ]:
# Create compelling funnel analysis with storytelling elements
fig, ax = plt.subplots(figsize=(16, 12))

# Calculate detailed funnel metrics
funnel_stages = [
    ('Marketing Qualified Leads', len(df)),
    ('Sales Development Rep Contact', len(df[df['sdr_id'].notna()])),
    ('Sales Representative Contact', len(df[df['sr_id'].notna()])),
    ('Closed Won Deals', len(df[df['won_date'].notna()])),
    ('Revenue Generating Deals', len(df[df['declared_monthly_revenue'] > 0]))
]

stages = [item[0] for item in funnel_stages]
counts = [item[1] for item in funnel_stages]
conversion_rates = [100] + [counts[i]/counts[i-1]*100 for i in range(1, len(counts))]

# Create funnel visualization with connecting lines
y_positions = np.arange(len(stages))
bar_width = 0.6

# Gradient colors for funnel stages
funnel_colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, len(stages)))

# Create bars
bars = ax.barh(y_positions, counts, height=bar_width, 
              color=funnel_colors, alpha=0.8, edgecolor='black', linewidth=2)

# Add conversion rate labels between stages
for i in range(len(conversion_rates)-1):
    if i < len(stages)-1:
        # Calculate position for conversion rate label
        x_pos = (counts[i] + counts[i+1]) / 2
        y_pos = y_positions[i] - bar_width/2 - 0.1
        
        # Create styled text box for conversion rate
        conv_color = nigerian_colors['success'] if conversion_rates[i+1] >= 70 else nigerian_colors['warning']
        ax.text(x_pos, y_pos, f'{conversion_rates[i+1]:.1f}%\nconversion',
                ha='center', va='top', fontsize=11, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.4', fc=conv_color, alpha=0.8, edgecolor='black'),
                color='white')

# Add stage labels and counts
for i, (bar, stage, count, rate) in enumerate(zip(bars, stages, counts, conversion_rates)):
    # Stage name
    ax.text(-counts[i]*0.02, bar.get_y() + bar.get_height()/2,
            stage, ha='right', va='center', fontsize=13, fontweight='bold')
    
    # Count and percentage
    ax.text(bar.get_width() + bar.get_width()*0.02, 
            bar.get_y() + bar.get_height()/2,
            f'{int(count):,}\n({rate:.1f}% of MQLs)',
            ha='left', va='center', fontsize=11, fontweight='bold')

# Add strategic annotations with arrows
# 1. Highlight biggest drop-off point
dropoff_rates = [(conversion_rates[i] - conversion_rates[i+1]) for i in range(len(conversion_rates)-1)]
max_dropoff_idx = dropoff_rates.index(max(dropoff_rates))

ax.annotate(f'⚠️ Largest Drop-off:\n{max(dropoff_rates):.1f}% decrease',
            xy=(counts[max_dropoff_idx+1], y_positions[max_dropoff_idx+1]),
            xytext=(counts[max_dropoff_idx+1]*0.5, y_positions[max_dropoff_idx+1]+0.5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.8),
            fontsize=11, fontweight='bold', color='darkred',
            ha='center')

# 2. Highlight best performing stage
best_conv_idx = conversion_rates[1:].index(max(conversion_rates[1:])) + 1
ax.annotate(f'✅ Best Conversion:\n{max(conversion_rates[1:]):.1f}% rate',
            xy=(counts[best_conv_idx], y_positions[best_conv_idx]),
            xytext=(counts[best_conv_idx]*1.3, y_positions[best_conv_idx]),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            bbox=dict(boxstyle='round,pad=0.5', fc='lightgreen', alpha=0.8),
            fontsize=11, fontweight='bold', color='darkgreen',
            ha='center')

# 3. Add strategic insights panel
insights_text = f"""
🎯 KEY INSIGHTS

• Overall Funnel Efficiency: {conversion_rates[-1]:.1f}%
• Critical Bottleneck: {stages[max_dropoff_idx]} → {stages[max_dropoff_idx+1]}
• Optimization Opportunity: Improve {stages[max_dropoff_idx+1]} conversion

💡 RECOMMENDATIONS

1. Focus on {stages[max_dropoff_idx+1]} process optimization
2. Analyze successful patterns in {stages[best_conv_idx]} stage
3. Implement targeted training for sales team
"""

ax.text(0.98, 0.02, insights_text,
        transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round,pad=1', fc=nigerian_colors['primary'], 
                 alpha=0.9, edgecolor='black', linewidth=2),
        color='white', fontweight='bold', family='monospace')

# Customize chart appearance
ax.set_title('Marketing Conversion Funnel Analysis - Nigerian E-commerce Platform', 
             fontsize=18, fontweight='bold', pad=30, color=nigerian_colors['primary'])
ax.set_xlabel('Number of Leads/Deals', fontsize=14, fontweight='bold')

# Format x-axis
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x):,}'))
ax.set_xlim(0, max(counts) * 1.3)

# Remove y-axis ticks and labels for cleaner look
ax.set_yticks([])
ax.set_ylabel('')

# Add subtle grid
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add branding
ax.text(0.5, 0.98, 'Q4 2023 Performance Analysis',
        transform=ax.transAxes, fontsize=12,
        horizontalalignment='center', verticalalignment='top',
        style='italic', alpha=0.7)

plt.tight_layout()
plt.show()

# Print actionable insights
print("\n" + "="*80)
print("FUNNEL ANALYSIS - ACTIONABLE INSIGHTS")
print("="*80)
print(f"\n📊 FUNNEL PERFORMANCE:")
for i, (stage, count, rate) in enumerate(zip(stages, counts, conversion_rates)):
    print(f"   {i+1}. {stage}: {count:,} ({rate:.1f}% of previous stage)")

print(f"\n🚨 CRITICAL BOTTLENECK:")
print(f"   {stages[max_dropoff_idx]} → {stages[max_dropoff_idx+1]}")
print(f"   {max(dropoff_rates):.1f}% drop-off rate")

print(f"\n💡 IMMEDIATE ACTIONS:")
print(f"   1. Analyze why {max(dropoff_rates):.1f}% drop at {stages[max_dropoff_idx+1]}")
print(f"   2. Implement process improvements for {stages[max_dropoff_idx+1]}")
print(f"   3. Study successful patterns in {stages[best_conv_idx]} (best converter)")
print("="*80)

## 3. Time Series Storytelling - Marketing Campaign Trends

Create compelling time-based narratives that show marketing performance evolution.

In [ ]:
# Create comprehensive time series analysis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 14))
fig.suptitle('Marketing Campaign Performance Trends - Nigerian Market Analysis', 
             fontsize=20, fontweight='bold', y=0.95)

# Calculate monthly metrics by channel
monthly_channel_metrics = df.groupby([df['first_contact_date'].dt.to_period('M'), 'origin']).agg({
    'mql_id': 'count',
    'won_date': lambda x: x.notna().sum(),
    'declared_monthly_revenue': 'sum'
}).rename(columns={
    'mql_id': 'mql_count',
    'won_date': 'converted_count'
}).reset_index()

monthly_channel_metrics['month'] = monthly_channel_metrics['first_contact_date'].dt.to_timestamp()
monthly_channel_metrics['conversion_rate'] = (monthly_channel_metrics['converted_count'] / 
                                              monthly_channel_metrics['mql_count'] * 100)

# 1. Top plot: MQL Volume Trends by Channel
channels = df['origin'].unique()
for channel in channels:
    channel_data = monthly_channel_metrics[monthly_channel_metrics['origin'] == channel]
    if not channel_data.empty:
        ax1.plot(channel_data['month'], channel_data['mql_count'],
                marker='o', linewidth=3, markersize=8,
                color=channel_colors.get(channel, nigerian_colors['info']),
                label=channel.title(), alpha=0.8)

# Add trend line for overall performance
overall_monthly = monthly_channel_metrics.groupby('month')['mql_count'].sum()
z = np.polyfit(range(len(overall_monthly)), overall_monthly.values, 1)
p = np.poly1d(z)
ax1.plot(overall_monthly.index, p(range(len(overall_monthly))), 
        "r--", alpha=0.6, linewidth=2, label='Overall Trend')

# Highlight peak and trough periods
peak_month = overall_monthly.idxmax()
trough_month = overall_monthly.idxmin()

ax1.annotate(f'🔥 Peak: {overall_monthly.max():,.0f} MQLs',
            xy=(peak_month, overall_monthly.max()),
            xytext=(10, 20), textcoords='offset points',
            bbox=dict(boxstyle='round,pad=0.5', fc=nigerian_colors['success'], alpha=0.7),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
            fontweight='bold', fontsize=11)

ax1.annotate(f'📉 Trough: {overall_monthly.min():,.0f} MQLs',
            xy=(trough_month, overall_monthly.min()),
            xytext=(10, -30), textcoords='offset points',
            bbox=dict(boxstyle='round,pad=0.5', fc=nigerian_colors['warning'], alpha=0.7),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
            fontweight='bold', fontsize=11)

ax1.set_title('Monthly MQL Generation by Marketing Channel', fontsize=16, fontweight='bold', pad=20)
ax1.set_ylabel('Number of MQLs', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98))
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x):,}'))

# 2. Bottom plot: Conversion Rate Evolution
for channel in channels:
    channel_data = monthly_channel_metrics[monthly_channel_metrics['origin'] == channel]
    if not channel_data.empty:
        ax2.plot(channel_data['month'], channel_data['conversion_rate'],
                marker='s', linewidth=3, markersize=8,
                color=channel_colors.get(channel, nigerian_colors['info']),
                label=channel.title(), alpha=0.8)

# Add performance threshold lines
ax2.axhline(y=10, color=nigerian_colors['success'], linestyle='--', alpha=0.7, 
          linewidth=2, label='Target (10%)')
ax2.axhline(y=5, color=nigerian_colors['danger'], linestyle='--', alpha=0.7,
          linewidth=2, label='Minimum (5%)')

# Highlight best performing channel
best_avg_channel = monthly_channel_metrics.groupby('origin')['conversion_rate'].mean().idxmax()
best_avg_rate = monthly_channel_metrics.groupby('origin')['conversion_rate'].mean().max()

ax2.annotate(f'🏆 Best Performer:\n{best_avg_channel.title()}\nAvg: {best_avg_rate:.1f}%',
            xy=(0.98, 0.95), xycoords='axes fraction',
            bbox=dict(boxstyle='round,pad=0.7', fc=nigerian_colors['primary'], 
                     alpha=0.9, edgecolor='black', linewidth=2),
            fontsize=11, fontweight='bold', color='white',
            ha='right', va='top')

ax2.set_title('Monthly Conversion Rate Trends', fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('Month', fontsize=14, fontweight='bold')
ax2.set_ylabel('Conversion Rate (%)', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98))
ax2.grid(True, alpha=0.3, linestyle='--')

# Format dates on x-axes
for ax in [ax1, ax2]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Print trend analysis insights
print("\n" + "="*80)
print("MARKETING TREND ANALYSIS - STRATEGIC INSIGHTS")
print("="*80)

# Calculate growth metrics
first_month = overall_monthly.iloc[0]
last_month = overall_monthly.iloc[-1]
growth_rate = ((last_month - first_month) / first_month) * 100

print(f"\n📈 OVERALL PERFORMANCE:")
print(f"   Period Growth: {growth_rate:+.1f}% (from {first_month:,} to {last_month:,} MQLs)")
print(f"   Volatility (CV): {overall_monthly.std()/overall_monthly.mean()*100:.1f}%")

print(f"\n🎯 CHANNEL PERFORMANCE:")
channel_insights = monthly_channel_metrics.groupby('origin').agg({
    'mql_count': 'sum',
    'conversion_rate': 'mean'
}).round({'conversion_rate': 1})

for channel, metrics in channel_insights.iterrows():
    print(f"   {channel.title()}: {metrics['mql_count']:,} MQLs, {metrics['conversion_rate']:.1f}% avg conversion")

print(f"\n💡 STRATEGIC RECOMMENDATIONS:")
if growth_rate > 0:
    print(f"   ✅ Positive growth trend ({growth_rate:+.1f}%) - continue current strategy")
else:
    print(f"   ⚠️ Declining trend ({growth_rate:+.1f}%) - requires immediate attention")

print(f"   🎯 Focus resources on {best_avg_channel} (best converter: {best_avg_rate:.1f}%)")
print(f"   📊 Analyze peak factors from {peak_month.strftime('%B %Y')} for replication")
print("="*80)

## 4. Executive Presentation - The Art of the Ask

Create boardroom-ready visualizations that drive decision-making and budget approval.

In [ ]:
# Create executive summary dashboard with ROI focus
fig = plt.figure(figsize=(24, 16))
fig.suptitle('EXECUTIVE DASHBOARD: Marketing ROI & Performance Analysis', 
             fontsize=24, fontweight='bold', y=0.95, color=nigerian_colors['primary'])

# Create complex grid layout
gs = fig.add_gridspec(4, 4, hspace=0.4, wspace=0.3)

# Calculate ROI metrics (assuming estimated costs for demo purposes)
estimated_costs = {
    'social': 500000,
    'paid_search': 800000,
    'organic_search': 200000,
    'email': 150000,
    'referral': 100000
}

roi_data = df.groupby('origin').agg({
    'mql_id': 'count',
    'won_date': lambda x: x.notna().sum(),
    'declared_monthly_revenue': 'sum'
}).reset_index()

roi_data['estimated_cost'] = roi_data['origin'].map(estimated_costs)
roi_data['roi'] = ((roi_data['declared_monthly_revenue'] - roi_data['estimated_cost']) / 
                   roi_data['estimated_cost'] * 100)
roi_data['cpa'] = roi_data['estimated_cost'] / roi_data['won_date']  # Cost per acquisition

# 1. Header ROI Summary (Top row, spanning all columns)
ax_header = fig.add_subplot(gs[0, :])
ax_header.axis('off')

# Calculate key metrics
total_revenue = roi_data['declared_monthly_revenue'].sum()
total_cost = roi_data['estimated_cost'].sum()
total_roi = ((total_revenue - total_cost) / total_cost) * 100
total_conversions = roi_data['won_date'].sum()
avg_cpa = total_cost / total_conversions

executive_summary = f"""
📊 MARKETING PERFORMANCE SNAPSHOT

💰 Total Revenue Generated: ₦{total_revenue:,.0f}  |  💸 Total Investment: ₦{total_cost:,.0f}
📈 Overall ROI: {total_roi:+.1f}%  |  🎯 Total Conversions: {total_conversions:,}
💳 Average Cost per Acquisition: ₦{avg_cpa:,.0f}  |  🚀 Performance Period: Q4 2023

🏆 Top Performing Channel: {roi_data.loc[roi_data['roi'].idxmax(), 'origin'].title()}
⚡ Best ROI: {roi_data['roi'].max():+.1f}%
"""

ax_header.text(0.5, 0.5, executive_summary,
              transform=ax_header.transAxes, fontsize=14, fontweight='bold',
              ha='center', va='center', fontfamily='monospace',
              bbox=dict(boxstyle='round,pad=1.5', fc=nigerian_colors['primary'], 
                       alpha=0.1, edgecolor=nigerian_colors['primary'], linewidth=3))

# 2. ROI by Channel (Left side)
ax_roi = fig.add_subplot(gs[1, :2])
roi_sorted = roi_data.sort_values('roi', ascending=True)
roi_colors = [nigerian_colors['success'] if roi > 50 else 
              nigerian_colors['warning'] if roi > 0 else 
              nigerian_colors['danger'] for roi in roi_sorted['roi']]

bars_roi = ax_roi.barh(roi_sorted['origin'], roi_sorted['roi'], 
                      color=roi_colors, alpha=0.8, edgecolor='black', linewidth=2)
ax_roi.set_title('Return on Investment (ROI) by Channel', fontsize=16, fontweight='bold', pad=15)
ax_roi.set_xlabel('ROI (%)', fontsize=14, fontweight='bold')
ax_roi.axvline(x=0, color='black', linestyle='-', linewidth=2)
ax_roi.grid(axis='x', alpha=0.3, linestyle='--')

# Add ROI labels
for bar, roi in zip(bars_roi, roi_sorted['roi']):
    width = bar.get_width()
    ax_roi.text(width + (5 if width >= 0 else -25), bar.get_y() + bar.get_height()/2,
                f'{roi:+.1f}%', ha='left' if width >= 0 else 'right', 
                va='center', fontweight='bold', fontsize=11)

# 3. Cost per Acquisition (Right side)
ax_cpa = fig.add_subplot(gs[1, 2:])
cpa_sorted = roi_data.sort_values('cpa', ascending=True)

bars_cpa = ax_cpa.barh(cpa_sorted['origin'], cpa_sorted['cpa'], 
                      color=nigerian_colors['gradient'][2], alpha=0.8, 
                      edgecolor='black', linewidth=2)
ax_cpa.set_title('Cost per Acquisition (CPA) by Channel', fontsize=16, fontweight='bold', pad=15)
ax_cpa.set_xlabel('Cost per Acquisition (₦)', fontsize=14, fontweight='bold')
ax_cpa.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'₦{int(x/1000)}K'))
ax_cpa.grid(axis='x', alpha=0.3, linestyle='--')

# Add CPA labels
for bar, cpa in zip(bars_cpa, cpa_sorted['cpa']):
    width = bar.get_width()
    ax_cpa.text(width + width*0.02, bar.get_y() + bar.get_height()/2,
                f'₦{cpa:,.0f}', ha='left', va='center', fontweight='bold', fontsize=11)

# 4. Revenue vs Cost Bubble Chart (Middle)
ax_bubble = fig.add_subplot(gs[2, :])
bubble_size = roi_data['won_date'] * 50  # Scale bubble size by number of conversions

scatter = ax_bubble.scatter(roi_data['estimated_cost'], roi_data['declared_monthly_revenue'], 
                          s=bubble_size, alpha=0.7, 
                          c=roi_data['roi'], cmap='RdYlGn', 
                          edgecolors='black', linewidth=2)

# Add breakeven line
max_cost = roi_data['estimated_cost'].max()
ax_bubble.plot([0, max_cost], [0, max_cost], 'r--', linewidth=2, 
               label='Breakeven Line', alpha=0.7)

# Add channel labels
for _, row in roi_data.iterrows():
    ax_bubble.annotate(row['origin'].title(), 
                      (row['estimated_cost'], row['declared_monthly_revenue']),
                      xytext=(5, 5), textcoords='offset points',
                      fontweight='bold', fontsize=10)

ax_bubble.set_title('Revenue vs Cost Analysis (Bubble Size = Conversions)', 
                  fontsize=16, fontweight='bold', pad=15)
ax_bubble.set_xlabel('Marketing Investment (₦)', fontsize=14, fontweight='bold')
ax_bubble.set_ylabel('Revenue Generated (₦)', fontsize=14, fontweight='bold')
ax_bubble.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'₦{int(x/1000)}K'))
ax_bubble.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'₦{int(x/1000)}K'))
ax_bubble.legend()
ax_bubble.grid(True, alpha=0.3)

# Add colorbar for ROI
cbar = plt.colorbar(scatter, ax=ax_bubble)
cbar.set_label('ROI (%)', rotation=270, labelpad=20, fontweight='bold')

# 5. Recommendations Box (Bottom)
ax_recs = fig.add_subplot(gs[3, :])
ax_recs.axis('off')

# Generate strategic recommendations
top_roi_channel = roi_data.loc[roi_data['roi'].idxmax(), 'origin']
lowest_cpa_channel = roi_data.loc[roi_data['cpa'].idxmin(), 'origin']
highest_revenue_channel = roi_data.loc[roi_data['declared_monthly_revenue'].idxmax(), 'origin']

recommendations = f"""
🎯 STRATEGIC RECOMMENDATIONS FOR Q1 2024 BUDGET ALLOCATION

✅ SCALE UP: {top_roi_channel.title()} (Best ROI: {roi_data['roi'].max():+.1f}%)
💡 OPTIMIZE: Improve underperforming channels with negative ROI
💰 REINVEST: Profits from {highest_revenue_channel.title()} into {lowest_cpa_channel.title()} (Best CPA)
📊 MONITOR: Weekly performance reviews and monthly budget adjustments

🚀 EXPECTED OUTCOMES:
   • 25% increase in overall ROI by reallocating budget to high-performers
   • 15% reduction in average CPA through channel optimization
   • ₦{(total_revenue * 0.25):,.0f} additional revenue with optimized budget allocation
"""

ax_recs.text(0.5, 0.5, recommendations,
             transform=ax_recs.transAxes, fontsize=13, fontweight='bold',
             ha='center', va='center', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=2', 
                      fc=nigerian_colors['warning'], alpha=0.2,
                      edgecolor=nigerian_colors['warning'], linewidth=3))

# Add timestamp and confidentiality
fig.text(0.99, 0.01, f'Confidential Executive Report | Generated: {datetime.now().strftime("%Y-%m-%d %H:%M")}', 
         ha='right', fontsize=10, alpha=0.7, style='italic')

plt.show()

# Print executive briefing
print("\n" + "="*100)
print("EXECUTIVE BRIEFING: MARKETING ROI ANALYSIS")
print("="*100)
print(f"\n💰 FINANCIAL PERFORMANCE:")
print(f"   Total Investment: ₦{total_cost:,.0f}")
print(f"   Revenue Generated: ₦{total_revenue:,.0f}")
print(f"   Net Profit: ₦{total_revenue - total_cost:,.0f}")
print(f"   Overall ROI: {total_roi:+.1f}%")

print(f"\n🏆 TOP PERFORMERS:")
print(f"   Best ROI: {top_roi_channel.title()} ({roi_data['roi'].max():+.1f}%)")
print(f"   Lowest CPA: {lowest_cpa_channel.title()} (₦{roi_data['cpa'].min():,.0f})")
print(f"   Highest Revenue: {highest_revenue_channel.title()} (₦{roi_data['declared_monthly_revenue'].max():,.0f})")

print(f"\n💡 IMMEDIATE ACTIONS:")
print(f"   1. Increase budget for {top_roi_channel.title()} by 30%")
print(f"   2. Analyze and fix underperforming channels")
print(f"   3. Implement CPA reduction strategies across all channels")
print(f"   4. Set up weekly performance monitoring dashboard")

print(f"\n📈 Q1 2024 PROJECTIONS:")
projected_revenue = total_revenue * 1.25
projected_roi = total_roi * 1.3
print(f"   Projected Revenue: ₦{projected_revenue:,.0f} (+25%)")
print(f"   Projected ROI: {projected_roi:+.1f}% (+30%)")
print("="*100)

## Summary: Business Storytelling with Data

In this advanced session, we've mastered:

### 🎨 **Strategic Color Psychology**
- Nigerian business-appropriate color palettes
- Channel-specific color coding for instant recognition
- Performance-based color indicators (green/yellow/red)

### 📝 **Compelling Narratives**
- Funnel analysis with strategic annotations
- Time series storytelling with trend insights
- Executive dashboards that drive decisions

### 💼 **Boardroom-Ready Presentations**
- ROI-focused visualizations for budget discussions
- Strategic recommendations backed by data
- Professional formatting and branding

### 🎯 **Key Principles for Nigerian Business Context**
1. **Cultural Relevance**: Use locally meaningful colors and business references
2. **Economic Focus**: Emphasize ROI, cost efficiency, and revenue impact
3. **Actionable Insights**: Every visualization should lead to a business decision
4. **Professional Presentation**: Clean, uncluttered designs suitable for executive meetings

### 🚀 **Advanced Techniques Covered**
- Multi-dimensional bubble charts
- Strategic annotations and callouts
- Complex grid layouts for comprehensive dashboards
- Color psychology and brand alignment
- Executive summary generation

### 💡 **Next Steps for Mastery**
1. Practice with real Nigerian business data
2. Develop templates for common marketing reports
3. Study executive presentation best practices
4. Learn interactive visualization tools (Tableau, Power BI)

### 🎊 **Congratulations!**
You now have the skills to create compelling, business-focused visualizations that tell powerful stories and drive strategic decisions in the Nigerian e-commerce landscape!